In [2]:
%pylab widget
%load_ext autoreload
%autoreload 2

# from anysis_functions import *
import fastcat.fastcat as fc

from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib


In [19]:
# mv_6x = np.load('/home/xcite/MV_CBCT/projections/mv_300_6x.npy')[100]
# mv_6x_air = np.load('/home/xcite/MV_CBCT/projections/mv_air.npy')[100]
# mv_25x = np.load('/home/xcite/MV_CBCT/projections/mv_phantom_25x.npy')[100]
# mv_25x_air = np.load('/home/xcite/MV_CBCT/projections/mv_air_25x.npy')[100]
# kv = np.load('/home/xcite/MV_CBCT/projections/kv_phantom.npy')[100]
# kv_air = np.load('/home/xcite/MV_CBCT/projections/kv_air.npy')[100]

# proj_6x = -np.log(mv_6x/mv_6x_air)
# proj_25x = -np.log(mv_25x/mv_25x_air)
# proj_kv = -np.log(kv/kv_air)

In [20]:
proj_6x = np.load('proj_6x.npy')
proj_25x = np.load('proj_25x.npy')
proj_kv = np.load('proj_kv.npy')

In [21]:
rad = 90
plt.figure()
plot(mean(proj_6x[50:150],0)[rad:-(rad+12)],color='cornflowerblue',label= '6 MV')
plot(mean(proj_25x[:100],0)[9+rad:-(rad+3)],color='darkorange',label='2.5 MV')
plot(mean(proj_kv[50:150],0)[11+rad:-(rad + 1)],'k',label='100 kVp')
legend()
xlabel('pixel')
ylabel('$ln(I/I_0)$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$ln(I/I_0)$')

In [25]:
# This is the varian OBI inherent filtration as defined in the manual
s2 = fc.calculate_spectrum(100, 12, 3, 50,monitor=None)
s2.attenuate(0.3,fc.get_mu(z=13))
s2.attenuate(0.089,fc.get_mu(z=22))
s2.x, s2.y = s2.get_points()

In [35]:
nviews = 2 
angles = np.linspace(0,np.pi*2,nviews)

phantom = fc.Catphan_404()
phantom.phan_map = ['air','water','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Acrylic'] 

spectra = ['Varian_truebeam_phasespace',
           'W_spectrum_25',
           'kV']
detectors = ['CuGOS-784-micrometer',
           'CuGOS-784-micrometer',
           'CsI-784-micrometer']

scats = []


angles = np.linspace(0,np.pi*2,2)        
# phantom = fc.Catphan_515()
s = fc.Spectrum()

for detector, spectrum in zip(detectors, spectra):
    
    if spectrum == 'kV':
        s = s2
    else:
        s.load(spectrum)

    kernel = fc.Kernel(s, detector)
    if spectrum == 'kV':
        phantom.return_projs(kernel,s,angles,det_on=True,mgy = 1000.0,bowtie=True)
    else: 
        phantom.return_projs(kernel,s,angles,det_on=True,mgy = 1000.0)
        
    scats.append(np.mean(phantom.proj[0],0))

(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1899804.3099668277 number of photons 0.010763158668836276
(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 2768706.8113217996 number of photons 0.01015508406165716
(19, 19, 19) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 19529760.92844292 number of photons 0.6675340524127561


In [36]:
figure()
for ii in range(len(scats)):
    
    plot((scats[ii])/10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
dist = np.linspace(-256*0.0784 - 0.0392,256*0.0784 - 0.0392, 512) # TODO: fix this gore!!
factor = (152/(np.sqrt(dist**2 + 152**2)))**3
flood_summed = factor*660

plt.figure()
plt.subplot(111)

dist_long = np.linspace(-256*0.0784 - 0.0392,256*0.0784 - 0.0392, 1024)

# The primary projection don't include any scatter should add the coherent and the incoherent scatter
plt.plot(dist,-0.83*np.log((scats[0][-1::-1])/flood_summed),linewidth = 1.0,color = 'cornflowerblue')

plt.plot(dist,-1.0*np.log(real_spec_CT/real_spec_air),'k-.',linewidth = 0.4,markersize=0.4)
plt.xlim([-20,20])
# plt.plot(dist_long,(np.mean(Air[63,200:220],0)/3)[-1::-1],'-',linewidth = 1,color = 'darkorange')
plt.plot(dist_long,-1*np.log((np.mean(Catphan2[63,270:290],0))[-1::-1]/(np.mean(Air[63,270:275],0))[-1::-1]),'-',linewidth = 1,color = 'darkorange')
plt.legend(['fastCAT 6 MV W', 'Topas 6 MV W','Varian Truebeam 6 MV W', 'Topas 6 MV Al'])
plt.xlabel('[cm]')
plt.ylabel('Energy Deposition [MeV]')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …